In [ ]:
# Install dependencies
!pip install ultralytics opencv-python pyyaml -q

# Download dataset
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/crack-seg.zip -q
!unzip -q crack-seg.zip

from ultralytics import YOLO

model = YOLO('yolov8m.pt')
results = model.train(
    data='crack-seg.yaml',
    epochs=25,
    imgsz=640,
    batch=32,
    device=0,
    patience=10
)

print(f"Model saved: {results.best}")


In [ ]:
import os
from pathlib import Path

# Find training results
runs_dir = Path('/content/runs/detect')
train_dirs = sorted(runs_dir.glob('train*'), key=os.path.getmtime, reverse=True)

# Show latest training
latest = train_dirs[0]
print(f"\nLatest training: {latest.name}")
print(f"  Location: {latest}\n")

# Check for best model
best_model = latest / 'weights' / 'best.pt'
if best_model.exists():
    size = best_model.stat().st_size / 1e6
    print(f"Best model: best.pt ({size:.1f} MB)")
    print(f" Full path: {best_model}")

# Show results.csv
results_csv = latest / 'results.csv'
if results_csv.exists():
    print(f"\nTraining metrics (results.csv):")
    with open(results_csv) as f:
        lines = f.readlines()
        print(f"  Total epochs: {len(lines) - 1}")

        # Show column names
        print(f"\n  Metrics tracked:")
        header = lines[0].strip().split(',')
        for i, col in enumerate(header[:12]):
            print(f"    {i+1}. {col}")

# Show all plots
plots = sorted(latest.glob('*.png'))
if plots:
    print(f"\nGenerated plots ({len(plots)} images):")
    for plot in plots:
        print(f"    • {plot.name}")

# Show directory structure
print(f"\nDirectory structure:")
for item in latest.iterdir():
    if item.is_dir():
        count = len(list(item.glob('*')))
        print(f"    {item.name}/ ({count} files)")
    else:
        size = item.stat().st_size
        if size > 1e6:
            size_str = f"{size/1e6:.1f} MB"
        elif size > 1e3:
            size_str = f"{size/1e3:.1f} KB"
        else:
            size_str = f"{size} B"
        print(f"    {item.name} ({size_str})")



In [ ]:
from google.colab import files
files.download('/content/runs/detect/train3/weights/best.pt')

In [ ]:
from pathlib import Path
from IPython.display import Image, display

train_dir = Path('/content/runs/detect/train3')

# 1. Main results plot
print("\nMAIN TRAINING RESULTS")
print("-" * 70)
display(Image(str(train_dir / 'results.png')))

# 2. Precision-Recall curves
print("\nPRECISION-RECALL ANALYSIS")
print("-" * 70)
print("Precision curve:")
display(Image(str(train_dir / 'BoxP_curve.png')))
print("\nRecall curve:")
display(Image(str(train_dir / 'BoxR_curve.png')))
print("\nF1-Score curve:")
display(Image(str(train_dir / 'BoxF1_curve.png')))

# 3. Confusion matrix
print("\nCONFUSION MATRIX")
print("-" * 70)
print("Raw counts:")
display(Image(str(train_dir / 'confusion_matrix.png')))
print("\nNormalized (%):")
display(Image(str(train_dir / 'confusion_matrix_normalized.png')))

# 4. Validation predictions
print("\nVALIDATION PREDICTIONS")

for pred_file in sorted(train_dir.glob('*_pred.jpg')):
    print(f"\n{pred_file.name}:")
    display(Image(str(pred_file)))

import csv

with open(train_dir / 'results.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

    print(f"{'Epoch':<6} {'Train Loss':<12} {'Val Loss':<12} {'mAP50':<10} {'mAP50-95':<10}")
    print("-" * 50)

    for row in rows:
        epoch = row.get('epoch', '?')
        train_loss = row.get('train/box_loss', '?')
        val_loss = row.get('val/box_loss', '?')
        map50 = row.get('metrics/mAP50(B)', '?')
        map5095 = row.get('metrics/mAP50-95(B)', '?')

        print(f"{epoch:<6} {train_loss:<12} {val_loss:<12} {map50:<10} {map5095:<10}")
